<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

<h1 align=center><font size = 5>FILTRAGEM COLABORATIVA</font></h1>

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.

### Índice

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li> <p><a href="#ref1">Aquisição dos dados</a></p> </li>
    <li> <p><a href="#ref2">Pré-processamento</a></p> </li>
    <li> <p><a href="#ref3">Filtragem colaborativa</a></p> </li>
</ul>
<p></p>
</div>
<br>

<hr>

<a id="ref1"></a>
# Aquisição dos dados

Para adquirir e extrair os dados, basta executar os seguintes scripts bash:

Conjunto de dados obtido de [GroupLens](http://grouplens.org/datasets/movielens/). Vamos baixar o conjunto de dados. Para baixar os dados, utilizaremos o `!wget`. Utilizaremos o `!wget` para baixar os dados a partir do IBM Object Storage.

__Você sabia?__ Ao utilizar o aprendizado de máquina, é provável que trabalhe com conjuntos de dados de grandes dimensões. Como empresa, onde você pode hospedar seus dados? A IBM está oferecendo uma oportunidade única para empresas, com 10 TB de armazenamento no IBM Cloud Object Storage: Inscreva-se agora gratuitamente: [Inscreva-se agora gratuitamente](http://cocl.us/ML0101EN-IBM-Offer-CC)

In [ ]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

Agora está tudo pronto para começarmos a trabalhar com os dados!

<hr>

<a id="ref2"></a>
# Pré-processamento

Primeiro, tiramos todas as importações do caminho:

In [ ]:
#Biblioteca para manipulação do dataframe
import pandas as pd
#Quanto às funções matemáticas, só precisaremos da função sqrt, então será a única que importaremos
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Agora, lemos cada arquivo em seus dataframes:

In [ ]:
#Armazenamos as informações dos filmes em um dataframe do pandas
movies_df = pd.read_csv('movies.csv')
#Armazenamos as informações do usuário em um dataframe do pandas
ratings_df = pd.read_csv('ratings.csv')

Agora vamos dar uma olhada em como cada um está organizado:

In [ ]:
#Head é uma função que obtém as primeiras N linhas de um dataframe. O valor predeterminado de N é 5.
movies_df.head()

Assim, cada filme tem um ID (identificador) único, um título com o ano de lançamento (que pode conter caracteres unicode) e vários gêneros diferentes no mesmo campo. Removemos o ano da coluna title (título) e o colocamos em sua própria coluna, utilizando a função [extract](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html#pandas.Series.str.extract) do Pandas, que é muito conveniente.

Removemos o ano da coluna __title (título)__, utilizando a função replace do Pandas e o armazenamos em uma nova coluna __year (ano)__.

In [ ]:
#Usamos expressões regulares para buscar um ano indicado entre parênteses
#Especificamos os parênteses para não criar conflito com filmes que têm anos em seus títulos
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Eliminamos os parênteses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Eliminamos os anos da coluna ‘title’
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Aplicamos a função strip para eliminar todos os espaços finais que possam ter aparecido
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

Vejamos o resultado!

In [ ]:
movies_df.head()

E agora vamos também descartar a coluna de gêneros, já que não precisaremos dela para esse sistema de recomendação em particular.

In [ ]:
#Descartamos a coluna de gêneros
movies_df = movies_df.drop('genres', 1)

Este é o dataframe final dos filmes:

In [ ]:
movies_df.head()

<br>

Em seguida, observamos o dataframe de classificações.

In [ ]:
ratings_df.head()

Cada linha do dataframe de classificações tem um ID de usuário associado a pelo menos um filme, uma classificação e uma marca de hora que indica quando o filme foi classificado. Não precisaremos da coluna de marca de hora, então a descartamos para economizar memória.

In [ ]:
#Drop remove uma linha ou coluna especificada de um dataframe
ratings_df = ratings_df.drop('timestamp', 1)

Observe como fica o Dataframe final de classificações:

In [ ]:
ratings_df.head()

<hr>

<a id="ref3"></a>
# Filtragem colaborativa

Agora é hora de começarmos nosso trabalho com sistemas de recomendação.

A primeira técnica que veremos se chama __Filtragem Colaborativa__, também conhecida como __Filtragem Usuário-Usuário__. Conforme seu nome alternativo sugere, essa técnica utiliza outros usuários para recomendar itens ao usuário de entrada. Tenta encontrar usuários que têm preferências e opiniões semelhantes às do usuário de entrada e, em seguida, recomenda ao usuário de entrada os itens de que esses outros usuários gostaram. Existem vários métodos para encontrar usuários semelhantes (alguns até utilizam o aprendizado de máquina) e o que utilizaremos aqui será baseado na __função de correlação de Pearson__.

<img src="https://ibm.box.com/shared/static/1ql8cbwhtkmbr6nge5e706ikzm5mua5w.png" width=800px>


O processo para criar um sistema de recomendação baseado no usuário é o seguinte:
- selecione um usuário com os filmes que assistiu
- a partir de como ele classificou os filmes, encontre os X vizinhos principais
- obtenha o registro de filmes vistos por esses vizinhos.
- calcule uma pontuação de similaridade utilizando alguma fórmula
- recomende os itens com a maior pontuação

Começamos criando um usuário de entrada ao quem recomendar filmes:

__Aviso:__ para acrescentar mais filmes, basta aumentar a quantidade de elementos em userInput. Fique à vontade para acrescentar mais filmes! Só não se esqueça de escrevê-los com letras maiúsculas e se um filme começar com a palavra “The”, como em “The Matrix”, escreva-o da seguinte maneira: ‘Matrix, The’.

In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

#### Adicionar um ID de filme ao usuário de entrada
Com o usuário de entrada completo, extraímos os IDs de seus filmes do dataframe de filmes e adicionamos esses IDs ao dataframe de entrada.

Uma maneira de fazê-lo é primeiro filtrar as linhas que contêm o título dos filmes do usuário de entrada e, em seguida, combinar esse subconjunto com o dataframe de entrada. Também descartamos as colunas desnecessárias do dataframe de entrada para economizar memória.

In [ ]:
#Filtramos os filmes pelo título
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Em seguida, combinamos para obter os IDs dos filmes. Está implicitamente combinando por título.
inputMovies = pd.merge(inputId, inputMovies)
#Descartamos do dataframe de entrada as informações que não utilizaremos
inputMovies = inputMovies.drop('year', 1)
#Dataframe de entrada final
#Se um filme adicionado antes não aparecer aqui, é possível que não esteja no 
#dataframe original ou tenha sido escrito de modo diferente. Verifique as letras maiúsculas.
inputMovies

#### Usuários que assistiram aos mesmos filmes
Agora, com os IDs do filmes em nosso dataframe de entrada, podemos obter um subconjunto de usuários que assistiram aos filmes de nosso dataframe de entrada e os classificaram.


In [ ]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

Agora agrupamos as filas com base nos IDs de usuário.

In [ ]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

Observemos um dos usuários, por exemplo, o que tem a ID de usuário nº 1130

In [ ]:
userSubsetGroup.get_group(1130)

Também vamos ordenar esses grupos de modo a dar maior prioridade aos usuários que têm a maior quantidade de filmes em comum com o usuário de entrada. Assim, a recomendação será mais refinada, pois não teremos que passar por cada um dos usuários.

In [ ]:
#Ordenamos para que os usuários com mais filmes em comum com o usuário de entrada tenham prioridade
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

Observemos o primeiro usuário

In [ ]:
userSubsetGroup[0:3]

#### Semelhanças dos usuários com o usuário de entrada
A seguir, vamos comparar todos os usuários (nem todos, na verdade!!!) com nosso usuário especificado e encontrar aquele que é mais semelhante.

Utilizaremos o __coeficiente de correlação de Pearson__ para descobrir o grau de semelhança de cada usuário em relação ao usuário de entrada. Esse coeficiente é utilizado para medir a força de uma associação linear entre duas variáveis. A fórmula para encontrar esse coeficiente entre os conjuntos X e Y com N valores pode ser vista na imagem abaixo.

Por que a correlação de Pearson?

A correlação de Pearson é invariante ao escalonamento, isto é, não se altera ao multiplicarem-se todos os elementos por uma constante diferente de zero ou somar-se qualquer constante a todos os elementos. Por exemplo: se houver dois vetores X e Y, então, pearson(X, Y) == pearson(X, 2 * Y + 3). Essa é uma propriedade muito importante nos sistemas de recomendação porque, por exemplo, dois usuários podem classificar duas séries de itens totalmente diferentes em termos de classificações absolutas, mas seriam usuários semelhantes (ou seja, com ideias semelhantes) com classificações semelhantes em diversas escalas.

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

Os valores fornecidos pela fórmula variam de r = -1 a r = 1, onde 1 indica que há uma correlação direta entre as duas entidades (significa uma correlação positiva perfeita) e -1 indica que há uma correlação negativa perfeita.

Em nosso caso, 1 significa que os dois usuários têm gostos semelhantes, enquanto -1 significa o contrário.

Selecionaremos um subconjunto de usuários entre os quais fazer iterações. Esse limite é imposto porque não queremos perder muito tempo passando por cada um dos usuários.

In [ ]:
userSubsetGroup = userSubsetGroup[0:100]

Agora, calculamos a correlação de Pearson entre o usuário de entrada e os grupos do subconjunto e a armazenamos em um dicionário, em que a chave é a ID do usuário e o valor é o coeficiente

In [ ]:
#Armazenamos a correlação de Pearson em um dicionário, em que a chave é o ID do usuário e o valor é o coeficiente
pearsonCorrelationDict = {}

#Para cada grupo de usuários em nosso subconjunto
for name, group in userSubsetGroup:
    #Começamos ordenando o usuário de entrada e o grupo de usuários atual para que depois os valores não se misturem
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Obtemos o N para a fórmula
    nRatings = len(group)
    #Obtemos as classificações dos filmes que ambos têm em comum
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #Depois as armazenamos em uma variável buffer temporária em formato de lista, a fim de facilitar futuros cálculos
    tempRatingList = temp_df['rating'].tolist()
    #Também colocamos em formato de lista as classificações do grupo de usuários atual
    tempGroupList = group['rating'].tolist()
    #Agora calculamos a correlação de Pearson entre dois usuários, que chamaremos de x e y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #Se o denominador for diferente de zero, dividimos, se não, a correlação é 0.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [ ]:
pearsonCorrelationDict.items()

In [ ]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

#### Os x usuários mais semelhantes ao usuário de entrada
Vamos obter os 50 usuários que são mais semelhantes ao usuário de entrada.

In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

Comecemos recomendando filmes ao usuário de entrada.

#### Classificações de todos os filmes dos usuários selecionados
Para fazer isso, vamos calcular a média ponderada das classificações dos filmes utilizando como peso a correlação de Pearson. Mas primeiro precisamos obter os filmes assistidos pelos usuários em nosso __pearsonDF__ a partir do dataframe de classificações e, em seguida, armazenar sua correlação em uma nova coluna denominada _“_similarityIndex”_. Para isso, combinamos essas duas tabelas.

In [ ]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

Agora, tudo o que precisamos fazer é multiplicar a classificação do filme por seu peso (o índice de similaridade), somar as novas classificações e dividir o resultado pela soma dos pesos.

Uma maneira fácil de fazê-lo é multiplicar duas colunas, agrupar o dataframe pelo ID do filme e, em seguida, dividir duas colunas:

Mostra a ideia de que todos os usuários semelhantes proponham filmes para o usuário de entrada:

In [ ]:
#Multiplica a similaridade pelas classificações dos usuários
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

In [ ]:
#Aplica uma soma aos usuários mais semelhantes (topUsers) após agrupá-los por ID de usuário (userId)
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

In [ ]:
#Cria um dataframe vazio
recommendation_df = pd.DataFrame()
#Agora, tomamos a média ponderada
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

Agora, vamos ordená-los e ver quais são os 20 filmes mais recomendados pelo algoritmo!

In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

In [ ]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

### Vantagens e desvantagens da filtragem colaborativa

##### Vantagens
* Leva em consideração as classificações de outros usuários
* Não necessita estudar ou extrair informações do item recomendado
* Adapta-se aos interesses do usuário, que podem mudar ao longo do tempo


##### Desvantagens
* A função de aproximação pode ser lenta
* Pode haver poucos usuários com os quais fazer a aproximação
* Problemas de privacidade ao tentar entender as preferências do usuário

## Deseja saber mais?

A IBM SPSS Modeler é uma plataforma analítica abrangente que possui muitos algoritmos de aprendizado de máquina. Foi projetada para levar inteligência preditiva às decisões tomadas pelas pessoas, pelos grupos, pelos sistemas e por sua empresa como um todo. Este curso lhe permite acessar uma avaliação gratuita, disponível aqui: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

Também é possível utilizar o Watson Studio para executar esses notebooks mais rapidamente com conjuntos de dados maiores. O Watson Studio é a solução de nuvem de vanguarda da IBM para cientistas de dados, construída por cientistas de dados. Com os notebooks Jupyter, RStudio, Apache Spark e outras bibliotecas populares pré-empacotadas na nuvem, o Watson Studio permite que os cientistas de dados colaborem em seus projetos sem a necessidade de instalar nada. Junte-se hoje mesmo à comunidade de usuários do Watson Studio, que cresce cada dia mais, com uma conta gratuita em [Watson Studio](https://cocl.us/ML0101EN_DSX)

### Obrigado por concluir esta lição!

Notebook criado por: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). Este notebook e seu código-fonte são divulgados de acordo com os termos da [Licença do MIT](https://bigdatauniversity.com/mit-license/).​